**Login to HuggingFace**

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install config -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.8 MB/s eta 0:00:00


**Import Dependencies**

In [3]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DistilBertForSequenceClassification  # For text classification


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**Import the Dataset**

In [11]:
data_path = "/content/drive/MyDrive/combined_data.csv"
df = pd.read_csv(data_path)

In [12]:
import pandas as pd

# Assuming your dataframe is called 'df'

# Two ways to achieve this:

# Method 1: Using rename() with a dictionary
df = df.rename(columns={'old_column_name': 'description'})  # Replace 'old_column_name' with your actual column name

# Method 2: Assigning directly to the 'columns' attribute
df.columns = ['description']

# Now 'df' has a single column named "description"


In [13]:
column_names = df.columns
print(column_names)

Index(['description'], dtype='object')


**Split the dataset into train and test**

In [14]:
from sklearn.model_selection import train_test_split

# Define the split ratio (e.g., 80% for training, 20% for testing)
train_size = 0.8
test_size = 1 - train_size

# Split the dataset
train, test = train_test_split(df, train_size=train_size, test_size=test_size)


**Import the model and tokenizer**

In [ ]:
from transformers import AutoModelForSequenceClassification
model_name = 'distilbert-base-uncased'  # Replace with your desired model variant
model = AutoModelForSequenceClassification.from_pretrained(model_name)

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

**Tokenizing**

In [16]:
# Assuming 'df' is your DataFrame and 'description' is the column with text data
tokenized_data = df['description'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True, return_tensors='pt'))


**2nd preprocessing function to**

*   Concatenate all the sequences
*   Split the concatenated sequences into shorter chunks defined by block_size, which should be both shorter than the maximum input length and short enough for your GPU RAM.



In [ ]:
block_size = 128


def group_texts(examples):
    concatenated_examples = {k: examples[k] + [] for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.

    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [19]:
import pandas as pd

# Replace the ellipsis with actual data
df = pd.DataFrame({
    'description': [tokenized_data],  # Replace with your actual data
    # ... other columns if any
})


block_size = 128

def group_texts(df):
    # Assuming 'input_ids' is the column with tokenized text data
    concatenated_examples = df['description'].sum()

    # Calculate the total length and adjust it to a multiple of block_size
    total_length = len(concatenated_examples)
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size

    # Split the concatenated input_ids into fixed-size blocks
    result = [concatenated_examples[i: i + block_size] for i in range(0, total_length, block_size)]

    # Return the result as a list of blocks
    return result

# Apply the function to the DataFrame
grouped_blocks = group_texts(df)


**Now create a batch of examples using DataCollatorForLanguageModeling. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.**

In [20]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")

**Train(Tensor Flow)**

In [21]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [22]:
from transformers import TFAutoModelForMaskedLM

model = TFAutoModelForMaskedLM.from_pretrained("distilroberta-base")

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFRobertaForMaskedLM.

All the weights of TFRobertaForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    my_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    my_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)